In [1]:
import pandas as pd
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA

In [2]:
path_data = "../data/"
df = pd.read_pickle(path_data + "CPS_2010_2019.pkl")

In [3]:
df.columns

Index(['YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'ASECFLAG', 'HFLAG',
       'ASECWTH', 'PERNUM', 'WTFINL', 'CPSIDP', 'ASECWT', 'AGE', 'SEX', 'RACE',
       'HISPAN', 'EMPSTAT', 'LABFORCE', 'OCC', 'OCC2010', 'IND', 'LNKFW1MWT',
       'PANLWT'],
      dtype='object')

In [4]:
df["IND"].unique()

array([8190,  170,    0, 7580,  770, 8790, 7870, 7470, 9480, 8270, 8370,
       5380, 1870, 3590, 7860, 7280, 7390,  590, 3580, 7780, 6890, 9380,
       8180, 8680, 4470, 7380, 7980, 7290, 3170, 2280, 8570, 3090, 8660,
       9370, 2170, 4690, 7590, 6070, 4790, 4970, 6695, 2890, 6870, 6990,
       5170, 6480, 3380, 7970, 8770, 4890, 9470, 3190, 4270, 3960, 8470,
       7070, 6180, 6170, 7370, 4770, 6270, 5180, 5580, 7690, 5370, 2370,
       2190, 4380, 4880, 3870, 4090, 2290, 6290, 9190, 4190, 8590, 5390,
       9590, 7680, 6780, 6670, 6970, 9170, 7270, 7190, 6590, 2980, 4370,
       7480, 3490, 6380, 4990, 4260, 8690, 9160, 5080, 8980, 3890, 4870,
        180, 7990, 7790, 9290, 6190, 7880, 2880, 7670, 8560, 4170, 7570,
       8090, 7770, 2380, 2670,  670, 3370, 6470, 6370, 2570, 4390, 1880,
       4290, 5670, 8170, 3990, 1680, 4080, 1280, 2780, 4680, 6080, 4670,
       5090,  470, 3570, 9080, 1990, 5590, 1590,  290, 8870, 2870, 2480,
        570,  580, 8780, 6690, 6770, 5270, 8070, 94

In [5]:
len(df["IND"].unique())

273

In [9]:
df_unemployment = pd.read_csv(path_data + "occ2010_cps_broad_class.csv")

In [10]:
occ_codes = list(df_unemployment["Code"])

(empstat > =20 & empstat < 30) over weighted sum of people who are in laborforce (labforce=2, or empstat>=10 & empstat <30 tried both) 

In [75]:
df_2019 = pd.read_pickle(path_data + "CPS_df_occmobpanel_2019_asec.pkl")

In [14]:
# CPS EMP_STAT codes which mark a person as employed or unemployed.
employment_codes = [10, 12]
unemployment_codes = [20, 21,22]

In [116]:

unemp = df_2010["WTFINL"][(df_2010["AGE"] >= 16) & (df_2010["EMPSTAT"].isin([20, 21,22]))].sum()
labor_force = df_2010["WTFINL"][(df_2010["AGE"] >= 16) & (df_2010["LABFORCE"] == 2)].sum()

NameError: name 'df_2010' is not defined

In [120]:
# description https://cps.ipums.org/cps-action/variables/EMPSTAT#codes_section
df_2016_jan = df[(df["YEAR"] == 2016) & (df["MONTH"] == 1)]
unemp = df_2016_jan["WTFINL"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(unemployment_codes))].sum()
emp = df_2016_jan["WTFINL"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(employment_codes))].sum()
print("here ", unemp, "BLS, ", 8309000)
print("here ", emp, "BLS, ")

here  8458354.0893 BLS,  8309000
here  149634383.8275 BLS,  8309000


In [121]:
# description https://cps.ipums.org/cps-action/variables/EMPSTAT#codes_section
df_2016_jan = df[(df["YEAR"] == 2016) & (df["MONTH"] == 1)]
unemp = df_2016_jan["PANLWT"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(unemployment_codes))].sum()
emp = df_2016_jan["PANLWT"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(employment_codes))].sum()
print("here ", unemp, "BLS, ", 8309000)
print("here ", emp, "BLS, ")

here  8028188.5015 BLS,  8309000
here  148602476.577 BLS,  8309000


In [16]:
df['Idx'] = df.index
df = df.sort_values(by = ['CPSIDP', 'Idx'])

In [141]:
def calculate_unemp_cps(df):
    """ Function that calculates occupation\industry transitions
    Args:
        df(DaraFrame): dataframe with cps _sorted_ by cpsid
    """
    # setting id's to compare when a transition happens
    # NOTE emp_stat_old initiated and employed
    person_id_old, emp_stat_old, occ_old, ind_old = 0, 10, 0, 0  
    # counters 
    dict_occ_unemp = {}
    dict_occ_emp = {}
    for occ in occ_codes:
        dict_occ_emp[occ] = 0
        dict_occ_unemp[occ] = 0
        
    dict_occ_emp[9999.0] = 0
    dict_occ_unemp[9999.0] = 0
    # counters for raw (i.e. not using person weights) transitions
    raw_transitions_occ, raw_transitions_ind, raw_transitions_both = 0, 0, 0
    # coutner for estimated transitions in the use population (using PANLWT)
    transitions_occ, transitions_ind, transitions_both = 0, 0, 0
    # counter for people remaining in same occ or industry
    remained_occ, remained_ind = 0, 0
    
    count = 0
    # iterate over rows (df is sorted by person id)
    for index, row in df.iterrows():
        person_id_new = row["CPSIDP"]
        # still same person as before
        if person_id_new == person_id_old:
            
            # check employment status
            emp_status_new = row["EMPSTAT"]
            occ_old = row["OCC2010"]
#             if emp_stat_old != emp_status_new:
                
            # if emp -> unemp add to unemp of emp of origin occ
            if emp_stat_old in employment_codes and emp_status_new in unemployment_codes:
                if np.isfinite(row["PANLWT"]):
                    dict_occ_unemp[occ_old] += row["PANLWT"]
                emp_stat_old = emp_status_new

            # if emp -> emp add to emp of target occ. Update occupation
            elif emp_stat_old in employment_codes and emp_status_new in employment_codes:
                occ_old = row["OCC2010"]
                if np.isfinite(row["PANLWT"]):
                    dict_occ_emp[occ_old] += row["PANLWT"]

                emp_stat_old = emp_status_new


            # if unemp -> emp  add to emp of target occ. 
            elif emp_stat_old in unemployment_codes and emp_status_new in employment_codes:
                occ_old = row["OCC2010"]
                if np.isfinite(row["PANLWT"]):
                    dict_occ_emp[occ_old] += row["PANLWT"]
                emp_stat_old = emp_status_new


            # if unemp -> unemp add to unemp of last occupation recorded
            elif emp_stat_old in unemployment_codes and emp_status_new in unemployment_codes:
                if np.isfinite(row["PANLWT"]):
                    dict_occ_unemp[occ_old] += row["PANLWT"]
                emp_stat_old = emp_status_new
                        
        else:
            person_id_old = person_id_new
            occ_old = row["OCC2010"]
            ind_old = row["IND"]
                        
    return dict_occ_emp, dict_occ_unemp

In [158]:
df['Idx'] = df.index
df = df.sort_values(by = ['CPSIDP', 'Idx'])

In [159]:
df_2016_jan = df[(df["YEAR"] == 2016) & (df["MONTH"].isin([1,2]))]

In [160]:
dict_emp_2016, dict_unemp_2016 = calculate_unemp_cps(df_2016_jan)

In [161]:
np.sum(list(dict_unemp_2016.values()))

8033515.2831

In [162]:
np.sum(list(dict_emp_2016.values()))

149482963.383

In [164]:
# description https://cps.ipums.org/cps-action/variables/EMPSTAT#codes_section
df_2016_jan = df[(df["YEAR"] == 2016) & (df["MONTH"] == 1)]
unemp = df_2016_jan["PANLWT"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(unemployment_codes))].sum()
emp = df_2016_jan["PANLWT"][(df_2016_jan["AGE"] >= 16) & (df_2016_jan["EMPSTAT"].isin(employment_codes))].sum()
print("here ", unemp, "BLS, ", 8309000)
print("here ", emp, "BLS, ")

here  8028188.5015 BLS,  8309000
here  148602476.577 BLS, 


In [165]:
df['Idx'] = df.index
df = df.sort_values(by = ['CPSIDP', 'Idx'])

In [166]:
dict_emp, dict_unemp = calculate_unemp_cps(df)

In [172]:
df_unemployment["unemp_2010_2019"] = df_unemployment["Code"].map(dict_unemp)
df_unemployment["emp_2010_2019"] = df_unemployment["Code"].map(dict_emp)

In [173]:
df_unemployment["unemp_rate"] = df_unemployment["unemp_2010_2019"]/(df_unemployment["unemp_2010_2019"] + df_unemployment["emp_2010_2019"])

In [174]:
df_unemployment.to_csv(path_data + "cps_10_19_unemp_rate.csv")

In [175]:
df_unemployment["unemp_rate"] 

0      0.017110
1      0.031385
2      0.037789
3      0.039371
4      0.025838
         ...   
453         NaN
454         NaN
455         NaN
456    1.000000
457         NaN
Name: unemp_rate, Length: 458, dtype: float64

In [100]:
dict_unemp_2016

{10: 120988.44880000001,
 20: 81901.8483,
 30: 83808.4687,
 100: 19298.1879,
 110: 50387.079000000005,
 120: 123531.9657,
 130: 33419.5642,
 140: 30019.6504,
 150: 23502.633100000003,
 160: 25887.2336,
 205: 112341.04360000003,
 220: 86631.0701,
 230: 55341.3414,
 300: 14012.0548,
 310: 114378.50090000001,
 320: 0,
 330: 0,
 350: 66824.84180000001,
 360: 0,
 410: 60937.2839,
 420: 35079.328499999996,
 430: 384911.68949999986,
 500: 8579.728500000001,
 510: 0,
 520: 25647.1566,
 530: 10555.1652,
 540: 19446.2798,
 560: 32179.122400000004,
 600: 6624.24,
 620: 90681.1127,
 700: 11321.5478,
 710: 61503.77800000001,
 720: 15367.7762,
 730: 47416.55329999999,
 800: 123853.95679999999,
 810: 11292.918099999999,
 820: 13000.842499999999,
 830: 0,
 840: 22751.791400000002,
 850: 46669.976800000004,
 860: 15548.109900000001,
 900: 0,
 910: 23445.185599999997,
 930: 4822.7839,
 940: 14816.721899999999,
 950: 11865.428100000001,
 1000: 153521.74899999995,
 1010: 41056.1545,
 1020: 104076.7264,
 1

In [101]:
dict_emp_2016

{10: 36920.160299999996,
 20: 24655.2048,
 30: 23220.9385,
 100: 630.9293,
 110: 27972.8478,
 120: 51502.109200000006,
 130: 0,
 140: 926.8291,
 150: 4875.5652,
 160: 7137.689399999999,
 205: 14507.869200000001,
 220: 22493.9488,
 230: 11303.9945,
 300: 11815.685399999998,
 310: 49964.94580000001,
 320: 0,
 330: 0,
 350: 11850.5334,
 360: 0,
 410: 11372.3352,
 420: 9449.4251,
 430: 62709.13880000001,
 500: 0,
 510: 0,
 520: 8746.3864,
 530: 17394.0392,
 540: 4953.6141,
 560: 0,
 600: 0,
 620: 27488.5539,
 700: 0,
 710: 31607.515999999996,
 720: 13739.5062,
 730: 7112.8577,
 800: 81667.8291,
 810: 0,
 820: 0,
 830: 0,
 840: 11823.7776,
 850: 5213.214900000001,
 860: 0,
 900: 0,
 910: 11043.262700000001,
 930: 6169.2051,
 940: 4791.6855,
 950: 0,
 1000: 37922.584899999994,
 1010: 13491.2999,
 1020: 16098.6116,
 1050: 15285.0762,
 1060: 0,
 1100: 12546.095000000001,
 1200: 0,
 1220: 0,
 1230: 0,
 1240: 568.3722,
 1300: 12171.9082,
 1310: 0,
 1320: 0,
 1350: 0,
 1360: 0,
 1400: 0,
 1410: 9

In [98]:
dict_emp

{10: nan,
 20: nan,
 30: nan,
 100: nan,
 110: nan,
 120: nan,
 130: nan,
 140: nan,
 150: nan,
 160: nan,
 205: nan,
 220: nan,
 230: nan,
 300: 116472.65739999998,
 310: nan,
 320: 14629.6408,
 330: nan,
 350: nan,
 360: 13453.2041,
 410: nan,
 420: nan,
 430: nan,
 500: 211757.68479999996,
 510: 31554.2566,
 520: nan,
 530: nan,
 540: nan,
 560: 311305.3776999998,
 600: nan,
 620: nan,
 700: nan,
 710: nan,
 720: nan,
 730: nan,
 800: nan,
 810: 64094.0385,
 820: 90738.7863,
 830: 31703.7193,
 840: nan,
 850: nan,
 860: 94726.06120000001,
 900: 18975.0269,
 910: nan,
 930: 152704.4994,
 940: nan,
 950: nan,
 1000: nan,
 1010: nan,
 1020: nan,
 1050: nan,
 1060: 115013.79049999996,
 1100: nan,
 1200: 7480.1839,
 1220: nan,
 1230: 4318.2615000000005,
 1240: nan,
 1300: nan,
 1310: nan,
 1320: nan,
 1350: 108802.11220000002,
 1360: nan,
 1400: nan,
 1410: nan,
 1420: 45658.5943,
 1430: nan,
 1440: 26691.3824,
 1450: 74682.96589999998,
 1460: nan,
 1520: nan,
 1530: nan,
 1540: nan,
 15

In [97]:
df_unemployment["emp_2010_2019"]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
453    0.0
454    0.0
455    0.0
456    NaN
457    0.0
Name: emp_2010_2019, Length: 458, dtype: float64

In [96]:
df_unemployment["unemp_rate"]

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
453   NaN
454   NaN
455   NaN
456   NaN
457   NaN
Name: unemp_rate, Length: 458, dtype: float64

In [88]:
dict_unemp_2016_jan, dict_emp_2016_jan = calculate_unemp_cps(df_2016_jan)

In [90]:
dict_unemp_2016_jan

{10: 120988.44880000001,
 20: 81901.8483,
 30: 83808.4687,
 100: 19298.1879,
 110: 50387.079000000005,
 120: 123531.9657,
 130: 33419.5642,
 140: 30019.6504,
 150: 23502.633100000003,
 160: 25887.2336,
 205: 112341.04360000003,
 220: 86631.0701,
 230: 55341.3414,
 300: 14012.0548,
 310: 114378.50090000001,
 320: 0,
 330: 0,
 350: 66824.84180000001,
 360: 0,
 410: 60937.2839,
 420: 35079.328499999996,
 430: 384911.68949999986,
 500: 8579.728500000001,
 510: 0,
 520: 25647.1566,
 530: 10555.1652,
 540: 19446.2798,
 560: 32179.122400000004,
 600: 6624.24,
 620: 90681.1127,
 700: 11321.5478,
 710: 61503.77800000001,
 720: 15367.7762,
 730: 47416.55329999999,
 800: 123853.95679999999,
 810: 11292.918099999999,
 820: 13000.842499999999,
 830: 0,
 840: 22751.791400000002,
 850: 46669.976800000004,
 860: 15548.109900000001,
 900: 0,
 910: 23445.185599999997,
 930: 4822.7839,
 940: 14816.721899999999,
 950: 11865.428100000001,
 1000: 153521.74899999995,
 1010: 41056.1545,
 1020: 104076.7264,
 1

In [82]:
df_2019['Idx'] = df_2019.index
df_2019 = df_2019.sort_values(by = ['CPSIDP', 'Idx'])

In [83]:
df_2019["EMPSTAT"].value_counts()

10    661773
12     24023
21     22959
Name: EMPSTAT, dtype: int64

In [84]:
df["EMPSTAT"].value_counts()

10    7628593
0     3215559
36    2433903
34    1953380
32     733575
21     454469
12     288017
1       57353
22      48734
Name: EMPSTAT, dtype: int64

In [86]:
calculate_unemp_cps(df_2016_jan)

({10: 120988.44880000001,
  20: 81901.8483,
  30: 83808.4687,
  100: 19298.1879,
  110: 50387.079000000005,
  120: 123531.9657,
  130: 33419.5642,
  140: 30019.6504,
  150: 23502.633100000003,
  160: 25887.2336,
  205: 112341.04360000003,
  220: 86631.0701,
  230: 55341.3414,
  300: 14012.0548,
  310: 114378.50090000001,
  320: 0,
  330: 0,
  350: 66824.84180000001,
  360: 0,
  410: 60937.2839,
  420: 35079.328499999996,
  430: 384911.68949999986,
  500: 8579.728500000001,
  510: 0,
  520: 25647.1566,
  530: 10555.1652,
  540: 19446.2798,
  560: 32179.122400000004,
  600: 6624.24,
  620: 90681.1127,
  700: 11321.5478,
  710: 61503.77800000001,
  720: 15367.7762,
  730: 47416.55329999999,
  800: 123853.95679999999,
  810: 11292.918099999999,
  820: 13000.842499999999,
  830: 0,
  840: 22751.791400000002,
  850: 46669.976800000004,
  860: 15548.109900000001,
  900: 0,
  910: 23445.185599999997,
  930: 4822.7839,
  940: 14816.721899999999,
  950: 11865.428100000001,
  1000: 153521.7489999

In [65]:
df_2016_jan["EMPSTAT"].value_counts()

10    117678
0      49319
36     38456
34     31182
32     11977
21      5956
12      3459
1        839
22       553
Name: EMPSTAT, dtype: int64

In [66]:
calculate_unemp_cps(df_2016_jan)

emp status 10 0.0
person id 20141100000502.0
emp status 10 0.0
person id 20141100000503.0
emp status 10 36.0
person id 20141100001301.0
emp status 10 34.0
person id 20141100001302.0
emp status 10 32.0
person id 20141100002001.0
emp status 10 36.0
person id 20141100003101.0
emp status 10 0.0
person id 20141100003802.0
emp status 10 0.0
person id 20141100003803.0
emp status 10 0.0
person id 20141100003804.0
emp status 10 34.0
person id 20141100003805.0
emp status 10 34.0
person id 20141100003806.0
emp status 10 36.0
person id 20141100004501.0
emp status 10 32.0
person id 20141100004502.0
emp status 10 34.0
person id 20141100004503.0
emp status 10 0.0
person id 20141100004505.0
emp status 10 0.0
person id 20141100004506.0
emp status 10 32.0
person id 20141100005101.0
emp status 10 0.0
person id 20141100005802.0
emp status 10 0.0
person id 20141100005803.0
emp status 10 34.0
person id 20141100005804.0
emp status 10 0.0
person id 20141100008802.0


({10: 0,
  20: 0,
  30: 0,
  100: 0,
  110: 0,
  120: 0,
  130: 0,
  140: 0,
  150: 0,
  160: 0,
  205: 0,
  220: 0,
  230: 0,
  300: 0,
  310: 0,
  320: 0,
  330: 0,
  350: 0,
  360: 0,
  410: 0,
  420: 0,
  430: 0,
  500: 0,
  510: 0,
  520: 0,
  530: 0,
  540: 0,
  560: 0,
  600: 0,
  620: 0,
  700: 0,
  710: 0,
  720: 0,
  730: 0,
  800: 0,
  810: 0,
  820: 0,
  830: 0,
  840: 0,
  850: 0,
  860: 0,
  900: 0,
  910: 0,
  930: 0,
  940: 0,
  950: 0,
  1000: 0,
  1010: 0,
  1020: 0,
  1050: 0,
  1060: 0,
  1100: 0,
  1200: 0,
  1220: 0,
  1230: 0,
  1240: 0,
  1300: 0,
  1310: 0,
  1320: 0,
  1350: 0,
  1360: 0,
  1400: 0,
  1410: 0,
  1420: 0,
  1430: 0,
  1440: 0,
  1450: 0,
  1460: 0,
  1520: 0,
  1530: 0,
  1540: 0,
  1550: 0,
  1560: 0,
  1600: 0,
  1610: 0,
  1640: 0,
  1650: 0,
  1700: 0,
  1710: 0,
  1720: 0,
  1740: 0,
  1760: 0,
  1800: 0,
  1820: 0,
  1830: 0,
  1840: 0,
  1900: 0,
  1910: 0,
  1920: 0,
  1930: 0,
  1960: 0,
  1980: 0,
  2000: 0,
  2010: 0,
  2020: 0,
  20

In [30]:
df['Idx'] = df.index
df = df.sort_values(by = ['CPSIDP', 'Idx'])

In [31]:
df_2016_jan = df[(df["YEAR"] == 2016) & (df["MONTH"].isin([1,2]))]

In [37]:
df_2016_jan= df_2016_jan.sort_values(by = ['CPSIDP', 'Idx'])

In [40]:
df_2016_jan.columns

Index(['YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'ASECFLAG', 'HFLAG',
       'ASECWTH', 'PERNUM', 'WTFINL', 'CPSIDP', 'ASECWT', 'AGE', 'SEX', 'RACE',
       'HISPAN', 'EMPSTAT', 'LABFORCE', 'OCC', 'OCC2010', 'IND', 'LNKFW1MWT',
       'PANLWT', 'Idx'],
      dtype='object')

In [42]:
df["CPSIDP"].value_counts()

20140103299101    10
20130303059401    10
20171201779205    10
20130303056002    10
20130303056001    10
                  ..
20150906352803     1
20150906351106     1
20150906343604     1
20150906343603     1
20191207029104     1
Name: CPSIDP, Length: 2857826, dtype: int64

In [41]:
df[["CPSIDP", "YEAR", "MONTH", "EMPSTAT"]].head(50)

,CPSIDP,YEAR,MONTH,EMPSTAT
18328509,20081000001101,2010,1,10
18328510,20081000001102,2010,1,10
18328512,20081000001103,2010,1,32
18328511,20081000001104,2010,1,32
18306563,20081000001501,2010,1,36
18306564,20081000001502,2010,1,36
18357049,20081000002001,2010,1,36
18357050,20081000002002,2010,1,36
18328515,20081000004901,2010,1,10
18306558,20081000008901,2010,1,36


In [34]:
calculate_unemp_cps(df)

pass 0 10.0
pass 0 36.0
pass 0 34.0
pass 0 32.0
pass 0 10.0
pass 0 36.0
pass 0 10.0
pass 0 34.0
pass 0 34.0
pass 0 36.0
pass 0 32.0


({10: 0,
  20: 0,
  30: 0,
  100: 0,
  110: 0,
  120: 0,
  130: 0,
  140: 0,
  150: 0,
  160: 0,
  205: 0,
  220: 0,
  230: 0,
  300: 0,
  310: 0,
  320: 0,
  330: 0,
  350: 0,
  360: 0,
  410: 0,
  420: 0,
  430: 0,
  500: 0,
  510: 0,
  520: 0,
  530: 0,
  540: 0,
  560: 0,
  600: 0,
  620: 0,
  700: 0,
  710: 0,
  720: 0,
  730: 0,
  800: 0,
  810: 0,
  820: 0,
  830: 0,
  840: 0,
  850: 0,
  860: 0,
  900: 0,
  910: 0,
  930: 0,
  940: 0,
  950: 0,
  1000: 0,
  1010: 0,
  1020: 0,
  1050: 0,
  1060: 0,
  1100: 0,
  1200: 0,
  1220: 0,
  1230: 0,
  1240: 0,
  1300: 0,
  1310: 0,
  1320: 0,
  1350: 0,
  1360: 0,
  1400: 0,
  1410: 0,
  1420: 0,
  1430: 0,
  1440: 0,
  1450: 0,
  1460: 0,
  1520: 0,
  1530: 0,
  1540: 0,
  1550: 0,
  1560: 0,
  1600: 0,
  1610: 0,
  1640: 0,
  1650: 0,
  1700: 0,
  1710: 0,
  1720: 0,
  1740: 0,
  1760: 0,
  1800: 0,
  1820: 0,
  1830: 0,
  1840: 0,
  1900: 0,
  1910: 0,
  1920: 0,
  1930: 0,
  1960: 0,
  1980: 0,
  2000: 0,
  2010: 0,
  2020: 0,
  20

In [24]:
dict_occ_emp, dict_occ_unemp = calculate_unemp_cps(df_2016_jan)

({10: 0,
  20: 0,
  30: 0,
  100: 0,
  110: 0,
  120: 0,
  130: 0,
  140: 0,
  150: 0,
  160: 0,
  205: 0,
  220: 0,
  230: 0,
  300: 0,
  310: 0,
  320: 0,
  330: 0,
  350: 0,
  360: 0,
  410: 0,
  420: 0,
  430: 0,
  500: 0,
  510: 0,
  520: 0,
  530: 0,
  540: 0,
  560: 0,
  600: 0,
  620: 0,
  700: 0,
  710: 0,
  720: 0,
  730: 0,
  800: 0,
  810: 0,
  820: 0,
  830: 0,
  840: 0,
  850: 0,
  860: 0,
  900: 0,
  910: 0,
  930: 0,
  940: 0,
  950: 0,
  1000: 0,
  1010: 0,
  1020: 0,
  1050: 0,
  1060: 0,
  1100: 0,
  1200: 0,
  1220: 0,
  1230: 0,
  1240: 0,
  1300: 0,
  1310: 0,
  1320: 0,
  1350: 0,
  1360: 0,
  1400: 0,
  1410: 0,
  1420: 0,
  1430: 0,
  1440: 0,
  1450: 0,
  1460: 0,
  1520: 0,
  1530: 0,
  1540: 0,
  1550: 0,
  1560: 0,
  1600: 0,
  1610: 0,
  1640: 0,
  1650: 0,
  1700: 0,
  1710: 0,
  1720: 0,
  1740: 0,
  1760: 0,
  1800: 0,
  1820: 0,
  1830: 0,
  1840: 0,
  1900: 0,
  1910: 0,
  1920: 0,
  1930: 0,
  1960: 0,
  1980: 0,
  2000: 0,
  2010: 0,
  2020: 0,
  20

In [37]:
unemp

8310730.6959

In [ ]:
8309

In [ ]:
8309	8219

In [30]:
unemp

8310730.6959

In [31]:
labor_force

158804144.3182

In [24]:
unemp/labor_force

0.053502483420531335

179257823.02119997

In [5]:
df["AGE"].unique()

array([60, 29, 85, 52, 37, 75, 49, 59, 64, 68, 73, 55, 28, 14, 10,  8, 42,
       40, 76, 62, 45, 16, 33, 31,  0, 47, 26, 61, 80, 79, 51, 34, 44,  9,
        7, 69, 48,  5, 22, 23, 39,  4,  3, 24,  1, 58, 17, 38, 50, 36, 67,
       65, 53, 13, 11, 32, 12, 35, 25, 74, 43, 70,  6, 66, 71, 57, 20, 15,
       19, 78, 27, 30, 18, 63, 46,  2, 72, 54, 41, 77, 56, 21])